In [1]:
from datetime import datetime, date, timedelta
from typing import Dict, List
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.declarative import declared_attr
import uuid
from sqlalchemy.dialects.postgresql import UUID


app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)


In [ ]:
path=

In [ ]:
class User(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "user"

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String)
    level = db.Column(db.Integer)
    
    def __repr__(self):
        return '<User: {} | Name: {} | Level: {}>'.format(self.id, self.name, self.level)

    
    def update(self, data_changes):
        for key, val in data_changes.items():
            setattr(self, key, val)
        return self
    
    def update_diverging(self, data_changes):
        for k, v in data_changes.items():
            if data_changes[k] is not None and data_changes[k] != getattr(self, k):
                setattr(self, k, v)
        return self
    

In [ ]:
u1 = User(name='Thomas', level=5)
u2 = User(name='Tobi', level=2)

db.session.add(u1)
db.session.add(u2)
db.session.commit()

data_changes = {
    'name': 'Egon'
}

print(u1)
print(u2)

In [ ]:
u1.update_diverging(data_changes)
u2.update_diverging(data_changes)

db.session.commit()

In [ ]:
print(u1)
print(u2)

In [2]:
class Business(db.Model):  # type: ignore
    """ Business parent model """
    __tablename__ = "business"

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(24))
    created_on = db.Column(db.DateTime, default=datetime.utcnow)
    created_by = db.Column(db.Integer,  db.ForeignKey('user.id', name='fk_user_business_created_by'))
    employees = db.relationship('User', backref="employer", primaryjoin='User.employer_id==Business.id', cascade="all, delete-orphan")

    
    
class User(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "user"

    id = db.Column(db.Integer, primary_key=True)
    employer_id = db.Column(db.Integer, db.ForeignKey('business.id', name='fk_business_user_employer_id'))
    created_businesses = db.relationship('Business', backref='creator', order_by="desc(Business.created_on)", primaryjoin='Business.created_by==User.id', post_update=True)
    items = db.relationship('Item', backref="user", lazy='select')

    
    
class Item(db.Model):  # type: ignore
    """ User parent model """
    __tablename__ = "item"

    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id', name='fk_user_item_user_id'))
    
    

In [3]:
#db.reflect()
print("committing") #https://stackoverflow.com/questions/24289808/drop-all-freezes-in-flask-with-sqlalchemy
db.session.remove()

print("Dopping all")
db.drop_all()

print("Creating all")
db.create_all()

committing
Dopping all
Creating all


In [4]:
b1 = Business(name="b1")
b2 = Business(name="b2")

u1 = User()
u2 = User()

i1 = Item()
i2 = Item()

db.session.add(b1)
db.session.add(b2)
db.session.add(u1)
db.session.add(u2)
db.session.add(i1)
db.session.add(i2)

db.session.commit()

b1.employees.append(u1)
b1.employees.append(u2)
u1.created_businesses.append(b1)
u2.created_businesses.append(b2)
u1.items.append(i1)
u1.items.append(i2)
db.session.commit()

print(b1.employees)
print(u1.employer.name)
print(u1.created_businesses)
print(b2.created_by)
print(u1.items)


[<User 1>, <User 2>]
b1
[<Business 1>]
2
[<Item 1>, <Item 2>]


In [ ]:
db.session.delete(b1)
db.session.commit()

print("All Businesses:", Business.query.all())
print("All Employees:", User.query.all())
print("All Item:", Item.query.all())


In [6]:
len_items = Business.query.filter_by(id = 1).options(subqueryload(User.items)).all()

NameError: name 'subqueryload' is not defined

In [10]:
b1 = Business.query.filter_by(id=1).options(subqueryload(User.items)).all()

ArgumentError: Mapped attribute "User.items" does not apply to any of the root entities in this query, e.g. mapped class Business->business. Please specify the full path from one of the root entities to the target attribute. 

In [9]:
from sqlalchemy.orm import subqueryload

In [ ]:
q = db.session.query(Tag.text) # @new:modified
q = q.join(Tag.article)
q = q.join(Article.project)
q = q.filter(Project.id == project_id)
q = q.filter(Article.date_created.between(now-timedelta(hours=1), now))
tags = q.all()
return tags # @not

In [13]:
b1_items = Item.query.join(Item.user).join(User.employer).filter(Business.id == 1).all()

In [14]:
b1_items

[<Item 1>, <Item 2>]

In [7]:
#from datetime import datetime, date

In [8]:
import datetime

In [13]:
transaction_input_data = {'created_by': 3, 'bundle_id': 3, 'original_filename': 'Sales_Tax_Transaction_Report_M-2018-12.txt', 'account_id': 7, 'account_given_id': 'A2SC0NLSYTA68B', 'public_activity_period': '2018-DEC', 'channel_code': 'AFN', 'marketplace': 'amazon.de', 'transaction_type_public_code': 'SALE', 'given_id': '028-5913548-4610715', 'activity_id': 'DgxxgNB7Y', 'check_tax_calculation_date': None, 'shipment_date': datetime.date(2018, 12, 20), 'arrival_date': None, 'complete_date': datetime.date(2018, 12, 20), 'item_id': 264, 'item_sku': 'WE-01-02-01-10-30-109-44', 'item_name': 'weltenmann | Premium Herren Slip-on Espadrilles in Wildleder, Pebble beige 44', 'item_manufacture_country': None, 'item_quantity': 1, 'item_weight_kg': 0.99, 'item_weight_kg_total': 0.99, 'check_unit_cost_price_net': None, 'check_item_price_discount_net': None, 'check_item_price_discount_vat': None, 'item_price_discount_gross': 0.0, 'check_item_price_net': None, 'check_item_price_vat': None, 'item_price_gross': 105.0, 'check_item_price_total_net': None, 'check_item_price_total_vat': None, 'item_price_total_gross': 105.0, 'check_item_price_vat_rate': None, 'check_shipment_price_discount_net': None, 'check_shipment_price_discount_vat': None, 'shipment_price_discount_gross': 0.0, 'check_shipment_price_net': None, 'check_shipment_price_vat': None, 'shipment_price_gross': 0.0, 'check_shipment_price_total_net': None, 'check_shipment_price_total_vat': None, 'shipment_price_total_gross': 0.0, 'check_shipment_price_vat_rate': None, 'check_sale_total_value_net': None, 'check_sale_total_value_vat': None, 'sale_total_value_gross': 105.0, 'check_gift_wrap_price_discount_net': None, 'check_gift_wrap_price_discount_vat': None, 'gift_wrap_price_discount_gross': 0.0, 'check_gift_wrap_price_net': None, 'check_gift_wrap_price_vat': None, 'gift_wrap_price_gross': 0.0, 'check_gift_wrap_price_total_net': None, 'check_gift_wrap_price_total_vat': None, 'gift_wrap_price_total_gross': 0.0, 'check_gift_wrap_price_tax_rate': None, 'currency_code': 'EUR', 'check_item_tax_code_code': None, 'departure_country_code': 'DE', 'departure_postal_code': '36251', 'departure_city': 'Bad Hersfeld', 'arrival_country_code': 'DE', 'arrival_postal_code': '79801', 'arrival_city': 'HOHENTENGEN AM HOCHRHEIN', 'arrival_address': None, 'shipment_mode': 'CONSIGNMENT_BY_POST', 'shipment_conditions': 'DAP', 'check_departure_seller_vat_country_code': None, 'check_departure_seller_vat_number': None, 'check_arrival_seller_vat_country_code': None, 'check_arrival_seller_vat_number': None, 'check_seller_vat_country_code': None, 'check_seller_vat_number': None, 'check_tax_calculation_imputation_country': None, 'check_tax_jurisdiction': None, 'check_tax_jurisdiction_level': None, 'invoice_number': None, 'check_invoice_amount_vat': None, 'check_invoice_currency_code': None, 'check_invoice_exchange_rate': None, 'check_invoice_exchange_rate_date': None, 'invoice_url': None, 'check_export': False, 'customer_firm_name': None, 'customer_firm_vat_number': None, 'customer_firm_vat_number_country_code': None, 'supplier_vat_number': None, 'supplier_name': None}

In [14]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(transaction_input_data)

{   'account_given_id': 'A2SC0NLSYTA68B',
    'account_id': 7,
    'activity_id': 'DgxxgNB7Y',
    'arrival_address': None,
    'arrival_city': 'HOHENTENGEN AM HOCHRHEIN',
    'arrival_country_code': 'DE',
    'arrival_date': None,
    'arrival_postal_code': '79801',
    'bundle_id': 3,
    'channel_code': 'AFN',
    'check_arrival_seller_vat_country_code': None,
    'check_arrival_seller_vat_number': None,
    'check_departure_seller_vat_country_code': None,
    'check_departure_seller_vat_number': None,
    'check_export': False,
    'check_gift_wrap_price_discount_net': None,
    'check_gift_wrap_price_discount_vat': None,
    'check_gift_wrap_price_net': None,
    'check_gift_wrap_price_tax_rate': None,
    'check_gift_wrap_price_total_net': None,
    'check_gift_wrap_price_total_vat': None,
    'check_gift_wrap_price_vat': None,
    'check_invoice_amount_vat': None,
    'check_invoice_currency_code': None,
    'check_invoice_exchange_rate': None,
    'check_invoice_exchange_rate_d